# UrbanSoundsClassification: AIYard

Model.ipynb

AIYard

### 1. Kütüphanelerin içe aktarılması

In [13]:
import os
import numpy as np
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

### 2. Önişlenmiş verisetlerinin içe aktarılması
Önişleme defterini kullanarak `preproc` klasörü altında oluşturduğumuz `train`, `val` ve `test` klasörlerindeki spektrogramları okuyup demetler halinde kaydederek başlıyoruz.

#### 2.1. `train` klasöründeki spektrogramların okunması

X_train ve Y_train listelerini elde edeceğiz. Her iki listede 6900 tane eleman bulunmalı. 

X_train'e spektrogram verisi olan (64, 64) boyutlu NumPy dizilerini, Y_train'e ise karşılık gelen sınıfın numarasını ekleyeceğiz.

In [2]:
X_train = []
Y_train = []
for path, subdirs, files in os.walk("/home/ferit/repos/UrbanSoundsClassification/preproc/train"):
    for name in files:
        file = os.path.join(path, name)
        spectrogram = np.load(file)
        X_train.append(spectrogram)
        Y_train.append(int(file.split('-')[2][0]))

In [3]:
len(X_train) == len(Y_train) == 6900

True

#### 2.2. `val` klasöründeki spektrogramların okunması

X_val ve Y_val listelerini elde edeceğiz. Her iki listede 900 tane eleman bulunmalı.

In [4]:
X_val = []
Y_val = []
for path, subdirs, files in os.walk("/home/ferit/repos/UrbanSoundsClassification/preproc/val"):
    for name in files:
        file = os.path.join(path, name)
        spectrogram = np.load(file)
        X_val.append(spectrogram)
        Y_val.append(int(file.split('-')[2][0]))

In [6]:
len(X_val) == len(Y_val) == 900

True

#### 2.3. `test` klasöründeki spektrogramların okunması

X_test ve Y_test listelerini elde edeceğiz. Her iki listede 935 tane eleman bulunmalı.

In [5]:
X_test = []
Y_test = []
for path, subdirs, files in os.walk("/home/ferit/repos/UrbanSoundsClassification/preproc/test"):
    for name in files:
        file = os.path.join(path, name)
        spectrogram = np.load(file)
        X_test.append(spectrogram)
        Y_test.append(int(file.split('-')[2][0]))

In [9]:
len(X_test) == len(Y_test) == 935

True

### 3. Nöral ağın oluşturulması